In [54]:
import pandas as pa # type: ignore

df = pa.read_csv('MSJO_60_01-03_2023.csv', delimiter=';')

# Basis Datansatz sortieren
df_sorted = df.sort_values(by=['MESS_ID', 'VALUEDATE'], ascending=[True, True])
df_sorted.to_csv('sortierte_datei.csv', index=False)

# Drop rows with any missing values in MESS_ID, VALUEDATE, or PVALUE
df = df.dropna(subset=['MESS_ID', 'VALUEDATE', 'PVALUE'], how='any')

# Drop rows with Pvalue = 0
df = df[df['PVALUE'] != '0']

# Drop Duplicates
duplicates = df.duplicated().sum()
df.drop_duplicates(inplace=True)

# Set PVLAUE to float
df['PVALUE'] = df['PVALUE'].str.replace(',','.').astype(float)

# Quartile berechnen
Q1 = df['PVALUE'].quantile(0.25)
Q3 = df['PVALUE'].quantile(0.75)
IQR = Q3 - Q1

# Definiere die Schwellenwerte
obere_schwelle = Q3 + 1.5 * IQR
print(obere_schwelle)

# Filtere die Zeilen
df_filtered = df[(df['PVALUE'] >= obere_schwelle)]

# Gefiltertes DataFrame anzeigen
print("Gefiltertes DataFrame:")
print(df_filtered)

# Sort Cleand Dataset
df = df.sort_values(by=['MESS_ID', 'VALUEDATE'], ascending=[True, True])

# New CSV File
df.to_csv('cleaned.csv', index=False)
df_filtered.to_csv('test.csv', index=False)

# Calculate mean
pvalue_mean = df['PVALUE'].mean()







5.4825
Gefiltertes DataFrame:
        MESS_ID                    VALUEDATE     PVALUE
37194      5425  2023-01-10 15:00:00.0000000   9.208419
37195      5425  2023-01-10 15:01:00.0000000  11.302227
37196      5425  2023-01-10 15:02:00.0000000  11.289743
37197      5425  2023-01-10 15:03:00.0000000  11.369067
37198      5425  2023-01-10 15:04:00.0000000  11.272236
...         ...                          ...        ...
313512     5240  2023-03-13 08:38:00.0000000   5.880000
313513     5240  2023-03-13 08:39:00.0000000   6.000000
313514     5240  2023-03-13 08:40:00.0000000   5.880000
313515     5240  2023-03-13 08:41:00.0000000   5.640000
313516     5240  2023-03-13 08:42:00.0000000   5.520000

[11297 rows x 3 columns]
